# 02 - Data Preprocessing & Feature Engineering
## Wildfire Risk Prediction Project

This notebook handles data preprocessing, feature engineering, and data preparation for ML models.
We'll clean the data, create new features, and prepare train/test datasets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import joblib
import warnings
warnings.filterwarnings('ignore')

plt.style.use('dark_background')
print("🔧 Libraries loaded successfully!")

: 

## Load Dataset

In [ ]:
# Load the raw dataset
df = pd.read_csv('../backend/data/raw/wildfire_dataset.csv')

print(f"📊 Dataset loaded: {df.shape}")
print(f"📋 Columns: {list(df.columns)}")
print(f"\n🎯 Target variable distribution:")
print(df['occured'].value_counts())

df.head()

## Data Cleaning

In [ ]:
# Check for missing values
print("🔍 Missing Values Analysis:")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage
}).sort_values('Missing Count', ascending=False)

print(missing_df[missing_df['Missing Count'] > 0])

# Handle missing values if any
if missing_values.sum() > 0:
    print("\n🛠️ Handling missing values...")
    
    # Numeric columns: fill with median
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    imputer = SimpleImputer(strategy='median')
    df[numeric_columns] = imputer.fit_transform(df[numeric_columns])
    
    print("✅ Missing values handled with median imputation")
else:
    print("✅ No missing values found!")

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"🔍 Duplicate rows: {duplicates}")

if duplicates > 0:
    print(f"🧹 Removing {duplicates} duplicate rows...")
    df = df.drop_duplicates()
    print(f"✅ Dataset shape after removing duplicates: {df.shape}")

# Reset index
df = df.reset_index(drop=True)

## Outlier Detection and Treatment

In [ ]:
def detect_and_cap_outliers(df, column, method='iqr', cap_method='clip'):
    """
    Detect and handle outliers using IQR or Z-score method
    """
    if method == 'iqr':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
    elif method == 'zscore':
        mean = df[column].mean()
        std = df[column].std()
        lower_bound = mean - 3 * std
        upper_bound = mean + 3 * std
    
    # Count outliers
    outliers_count = ((df[column] < lower_bound) | (df[column] > upper_bound)).sum()
    
    if cap_method == 'clip' and outliers_count > 0:
        df[column] = np.clip(df[column], lower_bound, upper_bound)
    
    return outliers_count, lower_bound, upper_bound

# Apply outlier treatment to key features
key_features = ['temp_mean', 'humidity_min', 'wind_speed_max', 'pressure_mean', 'fire_weather_index']
outlier_summary = []

print("🎯 Outlier Detection and Treatment:")
for feature in key_features:
    if feature in df.columns:
        outliers_count, lower, upper = detect_and_cap_outliers(df, feature, method='zscore')
        outlier_summary.append({
            'Feature': feature,
            'Outliers Capped': outliers_count,
            'Lower Bound': f"{lower:.2f}",
            'Upper Bound': f"{upper:.2f}"
        })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df)
print(f"\n✅ Outliers handled using 3-sigma rule (Z-score method)")

## Feature Engineering

In [ ]:
print("🔬 Creating new features based on domain knowledge...")

# 1. Heat-Humidity Interaction
if 'temp_mean' in df.columns and 'humidity_min' in df.columns:
    df['heat_humidity_interaction'] = df['temp_mean'] * (100 - df['humidity_min']) / 100
    print("✅ Created heat_humidity_interaction feature")

# 2. Dryness-Wind Factor (fire spread potential)
if 'wind_speed_max' in df.columns and 'humidity_min' in df.columns:
    df['dryness_wind_factor'] = (100 - df['humidity_min']) * df['wind_speed_max'] / 100
    print("✅ Created dryness_wind_factor feature")

# 3. Temperature Range Features
if 'temp_mean' in df.columns and 'temp_range' in df.columns:
    df['temp_max_approx'] = df['temp_mean'] + df['temp_range'] / 2
    df['temp_min_approx'] = df['temp_mean'] - df['temp_range'] / 2
    print("✅ Created temperature range features")

# 4. Fire Weather Index Categories
if 'fire_weather_index' in df.columns:
    df['fwi_category'] = pd.cut(
        df['fire_weather_index'],
        bins=[-np.inf, 5, 10, 20, np.inf],
        labels=[1, 2, 3, 4]  # Low, Medium, High, Extreme
    )
    df['fwi_category'] = df['fwi_category'].astype(int)
    print("✅ Created FWI category feature")

# 5. Pressure Deviation (how far from standard pressure)
if 'pressure_mean' in df.columns:
    standard_pressure = 1013.25  # hPa at sea level
    df['pressure_deviation'] = abs(df['pressure_mean'] - standard_pressure)
    print("✅ Created pressure deviation feature")

# 6. Wind Direction Consistency
if 'wind_direction_std' in df.columns:
    df['wind_consistency'] = 1 / (df['wind_direction_std'] + 1)  # Higher value = more consistent
    print("✅ Created wind consistency feature")

# 7. Evaporation Rate Proxy
if 'evapotranspiration_total' in df.columns and 'humidity_min' in df.columns:
    df['evaporation_rate'] = df['evapotranspiration_total'] / (df['humidity_min'] + 1)
    print("✅ Created evaporation rate proxy")

print(f"\n📊 Dataset shape after feature engineering: {df.shape}")

In [ ]:
# Display correlation of new features with target
new_features = [
    'heat_humidity_interaction', 'dryness_wind_factor', 'temp_max_approx', 
    'temp_min_approx', 'fwi_category', 'pressure_deviation', 
    'wind_consistency', 'evaporation_rate'
]

existing_features = [f for f in new_features if f in df.columns]

if existing_features:
    feature_correlations = df[existing_features + ['occured']].corr()['occured'].abs().sort_values(ascending=False)
    
    print("\n🎯 New Feature Correlations with Target:")
    for feature in existing_features:
        corr = feature_correlations[feature]
        print(f"  • {feature}: {corr:.4f}")
    
    # Visualize correlations
    plt.figure(figsize=(10, 6))
    feature_correlations[existing_features].plot(kind='bar')
    plt.title('🎯 New Feature Correlations with Fire Occurrence')
    plt.xlabel('Features')
    plt.ylabel('Absolute Correlation')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## Feature Selection

In [ ]:
# Define features to exclude from modeling
exclude_features = [
    'occured',  # target variable
    'frp',      # fire radiative power - result of fire, not predictor
    'lat', 'lon'  # location - will be used separately for mapping
]

# Select features for modeling
feature_columns = [col for col in df.columns if col not in exclude_features]
print(f"🎯 Selected {len(feature_columns)} features for modeling:")
for i, feature in enumerate(feature_columns, 1):
    print(f"  {i:2d}. {feature}")

# Prepare feature matrix and target vector
X = df[feature_columns]
y = df['occured']

print(f"\n📊 Final dataset shapes:")
print(f"  Features (X): {X.shape}")
print(f"  Target (y): {y.shape}")
print(f"  Target distribution: {y.value_counts().to_dict()}")

## Data Splitting

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Maintain class distribution
)

print("✂️ Data splitting completed:")
print(f"  Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"  Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(df)*100:.1f}%)")

print(f"\n🎯 Class distribution:")
print(f"  Training - Fire: {y_train.sum()} ({y_train.mean()*100:.1f}%), No Fire: {(y_train == 0).sum()} ({(y_train == 0).mean()*100:.1f}%)")
print(f"  Test - Fire: {y_test.sum()} ({y_test.mean()*100:.1f}%), No Fire: {(y_test == 0).sum()} ({(y_test == 0).mean()*100:.1f}%)")

## Feature Scaling

In [ ]:
# Initialize and fit scaler on training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("⚖️ Feature scaling completed with StandardScaler")
print(f"  Mean of scaled training features: {X_train_scaled.mean().mean():.6f}")
print(f"  Std of scaled training features: {X_train_scaled.std().mean():.6f}")

# Show scaling statistics for key features
print(f"\n📊 Scaling Statistics (first 5 features):")
for feature in X_train.columns[:5]:
    original_mean = X_train[feature].mean()
    original_std = X_train[feature].std()
    scaled_mean = X_train_scaled[feature].mean()
    scaled_std = X_train_scaled[feature].std()
    
    print(f"  {feature}:")
    print(f"    Original: μ={original_mean:.3f}, σ={original_std:.3f}")
    print(f"    Scaled:   μ={scaled_mean:.3f}, σ={scaled_std:.3f}")

## Save Preprocessed Data

In [ ]:
# Create directory if it doesn't exist
import os
processed_dir = '../backend/data/processed/'
os.makedirs(processed_dir, exist_ok=True)

# Save training data
train_data = pd.concat([X_train_scaled, y_train], axis=1)
train_data.to_csv(processed_dir + 'wildfire_train.csv', index=False)
print(f"✅ Training data saved: {train_data.shape}")

# Save test data
test_data = pd.concat([X_test_scaled, y_test], axis=1)
test_data.to_csv(processed_dir + 'wildfire_test.csv', index=False)
print(f"✅ Test data saved: {test_data.shape}")

# Save complete processed dataset
all_scaled_X = pd.concat([X_train_scaled, X_test_scaled])
all_y = pd.concat([y_train, y_test])
complete_data = pd.concat([all_scaled_X, all_y], axis=1)
complete_data.to_csv(processed_dir + 'wildfire_processed.csv', index=False)
print(f"✅ Complete processed data saved: {complete_data.shape}")

# Save the scaler for future use
joblib.dump(scaler, processed_dir + 'feature_scaler.pkl')
print("✅ Scaler saved for future use")

# Save feature names
feature_info = {
    'feature_columns': list(feature_columns),
    'target_column': 'occured',
    'excluded_columns': exclude_features,
    'new_features': existing_features
}
joblib.dump(feature_info, processed_dir + 'feature_info.pkl')
print("✅ Feature information saved")

## Final Data Quality Check

In [ ]:
# Final quality checks
print("🔍 Final Data Quality Assessment:")
print("=" * 50)

# Check for any remaining issues
print(f"✓ Training data shape: {X_train_scaled.shape}")
print(f"✓ Test data shape: {X_test_scaled.shape}")
print(f"✓ No missing values in training: {X_train_scaled.isnull().sum().sum() == 0}")
print(f"✓ No missing values in test: {X_test_scaled.isnull().sum().sum() == 0}")
print(f"✓ Features properly scaled: {abs(X_train_scaled.mean().mean()) < 1e-10}")
print(f"✓ Class balance maintained: Train={y_train.mean():.3f}, Test={y_test.mean():.3f}")

# Feature importance preview (correlation-based)
feature_importance = X_train_scaled.corrwith(y_train).abs().sort_values(ascending=False)
print(f"\n🏆 Top 10 Most Correlated Features:")
for i, (feature, corr) in enumerate(feature_importance.head(10).items(), 1):
    print(f"  {i:2d}. {feature}: {corr:.4f}")

print(f"\n✅ Preprocessing completed successfully!")
print(f"📁 Processed files saved to: {processed_dir}")

## Visualization of Processed Data

In [ ]:
# Create visualization of processed data
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Distribution of scaled features (sample)
sample_features = X_train_scaled.columns[:4]
for i, feature in enumerate(sample_features):
    row, col = i // 2, i % 2
    axes[row, col].hist(X_train_scaled[feature], bins=50, alpha=0.7, density=True)
    axes[row, col].set_title(f'Distribution of Scaled {feature}')
    axes[row, col].set_xlabel('Scaled Value')
    axes[row, col].set_ylabel('Density')
    axes[row, col].axvline(0, color='red', linestyle='--', alpha=0.5)

plt.suptitle('📊 Sample Feature Distributions After Scaling', fontsize=16)
plt.tight_layout()
plt.show()

# 2. Correlation heatmap of top features
top_features = feature_importance.head(10).index.tolist()
correlation_matrix = X_train_scaled[top_features].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            annot=True, 
            mask=mask,
            cmap='RdYlBu_r', 
            center=0,
            square=True,
            fmt='.3f')
plt.title('🔗 Correlation Matrix of Top 10 Features')
plt.tight_layout()
plt.show()

print("📊 Visualization completed!")